In [1]:
import numpy as np
import json
import hashlib
import pandas as pd

# TF

In [9]:
tf_results = pd.read_json("./tf2onnx_conv_results.json")

## TF Incorrect Output

In [10]:
tf_results.loc[tf_results['max_abs_diff'] >= 1e-3].shape

(9, 11)

## TF Conv. Error

In [24]:
conv_err_count = 0
for i, row in tf_results.iterrows():
    for filepaths in row['file_path']:
        if ("/tf2onnx" in filepaths):
            # print(row['error_tuples'])
            conv_err_count += 1
            # print(row['error_des'])
conv_err_count

65

## TF ONNX RUNTIME

In [25]:
count = 0
indices = []
for i, row in tf_results.iterrows():
    for filepaths in row['file_path']:
        if ("onnxruntime" in filepaths):
            # print(row['error_tuples'], row['error'])
            indices.append(i)
            count += 1
            continue
print(count)
print(len(indices))

19
19


## TF Unsuccessful Conversion (HF)

In [27]:
tf_results.loc[tf_results['error'] == 1].shape[0] - conv_err_count

456

## TF Success

In [28]:
maybe_success = tf_results.loc[(tf_results['error'] == 0) & (tf_results['max_abs_diff'] < 1e-3)]
maybe_success.drop(indices).shape

(1212, 11)

65

In [162]:
tf_results.loc[(tf_results['error'] == 1)]

,model_id,error,error_des,output,error_type,file_path,line_number,error_tuples,max_abs_diff,model_type,tags
0,ml6team-mt5-small-german-finetune-mlsum,1,2023-02-19 20:46:21.457633: I tensorflow/core/...,[],"[ValueError, ValueError]",[tf2onnx/lib/python3.9/site-packages/transform...,"[288, 120]",[[tf2onnx/lib/python3.9/site-packages/transfor...,0.0,t5,"[pytorch, tf, t5, text2text-generation, de, da..."
1,facebook-s2t-small-covost2-es-en-st,1,2023-02-19 20:46:27.107736: I tensorflow/core/...,[],[KeyError],[tf2onnx/lib/python3.9/site-packages/transform...,[562],[[tf2onnx/lib/python3.9/site-packages/transfor...,0.0,speech_to_text,"[pytorch, tf, speech_to_text, automatic-speech..."
6,ibraheemmoosa-xlmindic-base-uniscript-soham,1,2023-02-19 20:48:16.843826: I tensorflow/core/...,[],"[ValueError, ValueError]",[tf2onnx/lib/python3.9/site-packages/transform...,"[288, 120]",[[tf2onnx/lib/python3.9/site-packages/transfor...,0.0,albert,"[pytorch, tf, jax, albert, text-classification..."
11,google-t5-v1_1-large,1,2023-02-19 20:50:14.787453: I tensorflow/core/...,[],"[ValueError, ValueError]",[tf2onnx/lib/python3.9/site-packages/transform...,"[288, 120]",[[tf2onnx/lib/python3.9/site-packages/transfor...,0.0,t5,"[pytorch, tf, jax, t5, text2text-generation, e..."
12,google-vit-base-patch16-224-in21k,1,2023-02-19 20:50:27.573344: I tensorflow/core/...,[],"[ValueError, ImportError]",[tf2onnx/lib/python3.9/site-packages/transform...,"[288, 997]",[[tf2onnx/lib/python3.9/site-packages/transfor...,0.0,vit,"[pytorch, tf, jax, vit, feature-extraction, da..."
...,...,...,...,...,...,...,...,...,...,...,...
1737,mesolitica-t5-small-finetuned-noisy-ms-en,1,2023-02-19 13:42:13.967176: I tensorflow/core/...,[],"[ValueError, ValueError]",[tf2onnx/lib/python3.9/site-packages/transform...,"[288, 120]",[[tf2onnx/lib/python3.9/site-packages/transfor...,0.0,t5,"[pytorch, tf, tensorboard, t5, text2text-gener..."
1748,ibraheemmoosa-xlmindic-base-uniscript,1,2023-02-19 13:49:03.415196: I tensorflow/core/...,[],"[ValueError, ValueError]",[tf2onnx/lib/python3.9/site-packages/transform...,"[288, 120]",[[tf2onnx/lib/python3.9/site-packages/transfor...,0.0,albert,"[pytorch, tf, jax, albert, text-classification..."
1750,bayartsogt-albert-mongolian,1,2023-02-19 13:49:42.702163: I tensorflow/core/...,[],"[ValueError, ValueError]",[tf2onnx/lib/python3.9/site-packages/transform...,"[288, 120]",[[tf2onnx/lib/python3.9/site-packages/transfor...,0.0,albert,"[pytorch, tf, albert, fill-mask, mn, arxiv:190..."
1752,google-t5-efficient-base-nl36,1,2023-02-19 13:50:56.852301: I tensorflow/core/...,[],"[ValueError, ValueError]",[tf2onnx/lib/python3.9/site-packages/tensorflo...,"[497, 221]",[[tf2onnx/lib/python3.9/site-packages/tensorfl...,0.0,t5,"[pytorch, tf, jax, t5, text2text-generation, e..."


In [202]:
len(tf_results['model_type'].unique())

58

In [190]:
tf_results.loc[(tf_results['model_type'] == "t5")]['error'].value_counts()

0    183
1    122
Name: error, dtype: int64

In [184]:
x = tf_results.loc[(tf_results['model_type'] == "t5")]

In [191]:
count = 0
ind = []
for i, row in x.iterrows():
    for filepaths in row['file_path']:
        if ("/tf2onnx" in filepaths):
            print(row['error_tuples'])
            count += 1
            ind.append(i)
            continue
            # print(row['error_des'])

[['tf2onnx/lib/python3.9/site-packages/tensorflow/python/framework/importer.py', 'ValueError', '497'], ['tf2onnx/lib/python3.9/site-packages/tf2onnx/tf_loader.py', 'ValueError', '221']]
[['tf2onnx/lib/python3.9/site-packages/tensorflow/python/framework/importer.py', 'ValueError', '497'], ['tf2onnx/lib/python3.9/site-packages/tf2onnx/tf_loader.py', 'ValueError', '221']]
[['tf2onnx/lib/python3.9/site-packages/tensorflow/python/framework/importer.py', 'ValueError', '497'], ['tf2onnx/lib/python3.9/site-packages/tf2onnx/tf_loader.py', 'ValueError', '221']]
[['tf2onnx/lib/python3.9/site-packages/tensorflow/python/framework/importer.py', 'ValueError', '497'], ['tf2onnx/lib/python3.9/site-packages/tf2onnx/tf_loader.py', 'ValueError', '221']]
[['tf2onnx/lib/python3.9/site-packages/tensorflow/python/framework/importer.py', 'ValueError', '497'], ['tf2onnx/lib/python3.9/site-packages/tf2onnx/tf_loader.py', 'ValueError', '221']]
[['tf2onnx/lib/python3.9/site-packages/tensorflow/python/framework/imp

In [197]:
y = x.drop(ind)

In [198]:
count = 0
indices = []
for i, row in y.iterrows():
    for filepaths in row['file_path']:
        if ("onnxruntime" in filepaths):
            # print(row['error_tuples'], row['error'])
            indices.append(i)
            count += 1
            continue
print(count)

1


In [200]:
y.loc[indices]

,model_id,error,error_des,output,error_type,file_path,line_number,error_tuples,max_abs_diff,model_type,tags
494,google-t5-efficient-tiny-nh1,0,2023-02-19 14:25:04.984778: I tensorflow/core/...,[/home/pjajal/tmp7oxjop9y/model.onnx],[ONNXRuntimeError],[tf2onnx/lib/python3.9/site-packages/onnxrunti...,[200],[[tf2onnx/lib/python3.9/site-packages/onnxrunt...,0.0,t5,"[pytorch, tf, jax, t5, text2text-generation, e..."


# PT

In [43]:
pt_results = pd.read_json("./pytorch_conv_results.json")
pt_results.shape

(1761, 11)

## PT Incorrect Output

In [44]:
pt_results.loc[pt_results['max_abs_diff'] >= 1e-3].shape

(11, 11)

## PT Conv Errs

In [45]:
conv_err_count = 0
for i, row in pt_results.iterrows():
    for filepaths in row['file_path']:
        if ("/torch/" in filepaths):
            conv_err_count += 1
            continue
conv_err_count

20

## PT ONNX Runtime

In [46]:
count = 0
indices = []
for i, row in pt_results.iterrows():
    for filepaths in row['file_path']:
        if ("onnxruntime" in filepaths):
            # print(row['error_tuples'], row['error'])
            indices.append(i)
            count += 1
            continue
print(count)
print(len(indices))

27
27


## PT Unsuccessful Conversion (HF)

In [47]:
pt_results.loc[pt_results['error'] == 1].shape[0] - conv_err_count

342

## PT Success

In [48]:
maybe_success = pt_results.loc[(pt_results['error'] == 0) & (pt_results['max_abs_diff'] < 1e-3)]
maybe_success.drop(indices).shape

(1361, 11)